In [1]:
import numpy as np
import pandas as pd
import os
from tensorflow.keras.preprocessing import image
from tensorflow.keras import applications
from tensorflow.keras import Model
from tensorflow.keras.utils import Sequence
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [2]:
data = pd.read_csv('data/train.csv')

In [3]:
data.shape

(7254, 27)

In [4]:
df = data.sample(n=3500)

* I only take 3500 images randomly instead of the whole dataset to avoid crashing and out or memory

In [5]:
img_width = 224
img_height = 224
path_img = 'D:/Projects/FullProject/Generating_Topics_Images/building_model/data/Images/'
X = []
for i in range(df.shape[0]):
    path = path_img + data['Id'][i] + '.jpg'
    img = image.load_img(path, target_size=(img_width, img_height, 3))
    img = image.img_to_array(img)
    img = img / 255.0
    X.append(img)

X = np.array(X)

In [6]:
X.shape

(3500, 224, 224, 3)

In [7]:
y = df.drop(['Id', 'Genre'], axis = 1)
y = y.to_numpy()
y.shape

(3500, 25)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 25, test_size = 0.1)

* The Train Test ratio is 90-10

In [9]:
X_train.shape

(3150, 224, 224, 3)

In [10]:
X_test.shape

(350, 224, 224, 3)

### Define Model

In [11]:
model = applications.ResNet50V2(weights='imagenet', 
                            include_top=False, 
                            input_shape=(img_width, img_height, 3),
                            classes=25)

In [12]:
model.summary()

Model: "resnet50v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 114, 114, 64) 0           conv1_conv[0][0]                 
_________________________________________________________________________________________

In [13]:
x = model.output
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(25, activation='sigmoid')(x)

model = Model(inputs=model.input, outputs=predictions)

### Training

In [14]:
callbacks = [EarlyStopping(monitor='val_loss', patience=5),
             ModelCheckpoint('model_270123.h5', save_best_only=True)]

In [15]:
model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics=['binary_accuracy'])
history = model.fit(X_train, y_train, batch_size=4, 
                    epochs=100,
                    validation_data=(X_test, y_test),
                   callbacks=callbacks)

Epoch 1/100
788/788 [==============================] - 39s 43ms/step - loss: 0.3599 - binary_accuracy: 0.8826 - val_loss: 0.3038 - val_binary_accuracy: 0.9053


c:\users\administrator\appdata\local\programs\python\python39\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/100
788/788 [==============================] - 32s 41ms/step - loss: 0.2757 - binary_accuracy: 0.9049 - val_loss: 0.2528 - val_binary_accuracy: 0.9055
Epoch 3/100
788/788 [==============================] - 32s 41ms/step - loss: 0.2505 - binary_accuracy: 0.9067 - val_loss: 0.2495 - val_binary_accuracy: 0.9051
Epoch 4/100
788/788 [==============================] - 32s 41ms/step - loss: 0.2480 - binary_accuracy: 0.9069 - val_loss: 0.2494 - val_binary_accuracy: 0.9073
Epoch 5/100
788/788 [==============================] - 32s 41ms/step - loss: 0.2471 - binary_accuracy: 0.9068 - val_loss: 0.2497 - val_binary_accuracy: 0.9053
Epoch 6/100
788/788 [==============================] - 33s 41ms/step - loss: 0.2470 - binary_accuracy: 0.9073 - val_loss: 0.2495 - val_binary_accuracy: 0.9073
Epoch 7/100
788/788 [==============================] - 32s 41ms/step - loss: 0.2469 - binary_accuracy: 0.9070 - val_loss: 0.2495 - val_binary_accuracy: 0.9053
Epoch 8/100
788/788 [=========================

In [40]:
img = image.load_img('data/img2.jpg', target_size=(224,224,3))
img = image.img_to_array(img)
img = img/255.0
img = img.reshape(1,224,224,3)

In [41]:
classes = data.columns[2:]
y_pred = model.predict(img)

In [42]:
genre =np.argsort(y_pred[0])[:-6:-1]
for i in range(5):
    print(classes[genre[i]])

Drama
Comedy
Romance
Action
Crime
